<a href="https://colab.research.google.com/github/arsalanmubeen/IBA/blob/main/DSPD_Homework_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This homework is based off a notebook. 
1. You will alter this as need for your results / questions. 
2. When you've successfully run them on a colab, download as an .ipynb and submit on the LMS.

Make certain you are on a GPU Runtime first, by going to Runtime and selecting "Change Runtime Type", and then choosing Hardware Accelerator as GPU.

In this homework, you will build an emotion classifier based on a Huggingface emotions dataset.

You will need to install 🤗 Transformers, numpy and 🤗 Datasets. Run the following three cells.

In [ ]:
!pip install datasets==1.3.0

In [ ]:
!pip install transformers==4.3.2

In [ ]:
!pip install numpy==1.20.1

In [ ]:
from datasets import load_dataset
emotions_dataset = load_dataset('go_emotions', 'simplified')

Reusing dataset go_emotions (/root/.cache/huggingface/datasets/go_emotions/simplified/0.0.0/ef1c18ea192c771555f1e0d638889dd5f1896255782c57c6a0b934d5f94f779e)


To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset.

In [ ]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [ ]:
# For simplicity, train model with single label for item.
restricted_dataset = emotions_dataset.filter(lambda x: len(x["labels"]) == 1 and 27 not in x['labels'])
print(len(restricted_dataset['train']))
show_random_elements(restricted_dataset['train'])

Loading cached processed dataset at /root/.cache/huggingface/datasets/go_emotions/simplified/0.0.0/ef1c18ea192c771555f1e0d638889dd5f1896255782c57c6a0b934d5f94f779e/cache-2c7fcb671b791198.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/go_emotions/simplified/0.0.0/ef1c18ea192c771555f1e0d638889dd5f1896255782c57c6a0b934d5f94f779e/cache-45c473c456172fb5.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/go_emotions/simplified/0.0.0/ef1c18ea192c771555f1e0d638889dd5f1896255782c57c6a0b934d5f94f779e/cache-8d2b6d6a9a13ae19.arrow


23485


,id,labels,text
0,ed77yvz,[26],Wtf Powerhouse Macomb is the gym I go to. I've never seen this dude there.
1,ee2tthj,[11],"[NAME] also slept with his dads mistress, [NAME] is weird"
2,ef27v46,[17],What’s the reference matter if it’s still insulting? Glad your degree is coming in handy for something.
3,ee95p64,[24],Sorry if this is ignorant but is that a wig?
4,ef8kha5,[25],"as a woman, I can tell ya it’s not uncommon for that to happen, haha. [NAME] make it extra sad though."
5,edpco0z,[25],"i’m running, hiding & avoiding this man. poor girl. her story is a sad one"
6,edc75el,[1],Never again lol
7,edpq0v6,[3],"I used to work with a guy that was always whining how tough his life was. My family this, my friends that. Very annoying"
8,edewrth,[22],"I understand this keeps many dairies afloat (source neighbourhood dairy guy, so not 100 per cent sure it’s accurate)."
9,efgacry,[4],"Yes, he's really pushing the Devils to new heights."



The emotions are provided as numeric labels. These are the actual orderings, beginning at 0 for admiration:

In [ ]:
labels = ["positive", "neutral"]
index_to_labels = {index: label for index, label in enumerate(labels)}  

## Preprocessing the data

Before we can feed those texts to our model, we need to preprocess them. This is done by a 🤗 Transformers `Tokenizer` which will (as the name indicates) tokenize the inputs (including converting the tokens to their corresponding IDs in the pretrained vocabulary) and put it in a format the model expects, as well as generate the other inputs that model requires.

To do all of this, we instantiate our tokenizer with the `AutoTokenizer.from_pretrained` method, which will ensure:

- we get a tokenizer that corresponds to the model architecture we want to use,
- we download the vocabulary used when pretraining this specific checkpoint.

That vocabulary will be cached, so it's not downloaded again the next time we run the cell.

In [ ]:
from transformers import AutoConfig, AutoModelForSequenceClassification, AutoTokenizer

model_name = 'distilbert-base-uncased'
config = AutoConfig.from_pretrained(model_name, num_labels=27)
model = AutoModelForSequenceClassification.from_config(config=config)
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

Im using roberta-base modle 

In [ ]:
from transformers import AutoConfig, AutoModelForSequenceClassification, AutoTokenizer


model_name = 'bert-base-cased'
config = AutoConfig.from_pretrained(model_name, num_labels=27)
model = AutoModelForSequenceClassification.from_config(config=config)
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

We pass along `use_fast=True` to the call above to use one of the fast tokenizers (backed by Rust) from the 🤗 Tokenizers library. Those fast tokenizers are available for almost all models, but if you got an error with the previous call, remove that argument.

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples['text'], truncation=True)


def regroup_labels_to_int(example):
    if example['labels']==[0] or  example['labels']==[1] or example['labels']==[4] or  example['labels']==[13] or example['labels']==[15] or example['labels']==[17] or example['labels']==[18] or example['labels']== [21] or example['labels']==[22] or example['labels']==[23] or example['labels']== [26] :
     example['labels'] = 0
    elif example['labels']==[6] or example['labels']==[7] or example['labels']==[8] or example['labels']==[19]:
     example['labels'] = 2
    else :
     example['labels'] = 1
    return example


encoded_dataset = restricted_dataset.map(preprocess_function, batched=True)
encoded_dataset = encoded_dataset.map(regroup_labels_to_int)

# omitting the neutral values
encoded_dataset = encoded_dataset.filter(lambda x : x['labels'] != 2 )

Loading cached processed dataset at /root/.cache/huggingface/datasets/go_emotions/simplified/0.0.0/ef1c18ea192c771555f1e0d638889dd5f1896255782c57c6a0b934d5f94f779e/cache-a4f24cf12adc8469.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/go_emotions/simplified/0.0.0/ef1c18ea192c771555f1e0d638889dd5f1896255782c57c6a0b934d5f94f779e/cache-5a170ecd0c669a7e.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/go_emotions/simplified/0.0.0/ef1c18ea192c771555f1e0d638889dd5f1896255782c57c6a0b934d5f94f779e/cache-7afa427b6764fee8.arrow


## Fine-tuning the model

To instantiate a `Trainer`, we will need to define two more things. The most important is the [`TrainingArguments`](https://huggingface.co/transformers/main_classes/trainer.html#transformers.TrainingArguments), which is a class that contains all the attributes to customize the training. It requires one folder name, which will be used to save the checkpoints of the model, and all other arguments are optional:

In [ ]:
from transformers import Trainer, TrainingArguments
metric_name = "accuracy"

args = TrainingArguments(
    "test-emotions",
    evaluation_strategy = "epoch",
    learning_rate= 2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=2,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
)

Here we set the evaluation to be done at the end of each epoch, tweak the learning rate, use the `batch_size` defined at the top of the notebook and customize the number of epochs for training, as well as the weight decay. Since the best model might not be the one at the end of training, we ask the `Trainer` to load the best model it saved (according to `metric_name`) at the end of training.

In [ ]:
metric = datasets.load_metric('accuracy')
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

Then we just need to pass all of this along with our datasets to the `Trainer`:

In [ ]:
validation_key = "validation"
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

You might wonder why we pass along the `tokenizer` when we already preprocessed our data. This is because we will use it once last time to make all the samples we gather the same length by applying padding, which requires knowing the model's preferences regarding padding (to the left or right? with which token?). The `tokenizer` has a pad method that will do all of this right for us, and the `Trainer` will use it. You can customize this part by defining and passing your own `data_collator` which will receive the samples like the dictionaries seen above and will need to return a dictionary of tensors.

We can now finetune our model by just calling the `train` method:

In [ ]:
import numpy as np
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Runtime,Samples Per Second
1,0.603300,0.434165,0.795437,2.867800,917.090000
2,0.405900,0.402489,0.818251,2.986900,880.514000


TrainOutput(global_step=1298, training_loss=0.4734741058114497, metrics={'train_runtime': 166.7224, 'train_samples_per_second': 7.785, 'total_flos': 601017676394040, 'epoch': 2.0})

We can check with the `evaluate` method that our `Trainer` did reload the best model properly (if it was not the last one). We also run a sample prediction to demonstrate the API:

In [ ]:
trainer.evaluate()

{'epoch': 2.0,
 'eval_accuracy': 0.8182509505703423,
 'eval_loss': 0.40248873829841614,
 'eval_runtime': 2.9839,
 'eval_samples_per_second': 881.383}

In [ ]:
prepared_input = tokenizer.prepare_seq2seq_batch(["I am very happy"], return_tensors='pt')
model = model.to('cpu')
model.eval()
model_output = model(**prepared_input)
prediction = np.argmax(model_output.logits[0].detach().numpy())
index_to_labels[prediction]

'positive'

In [ ]:
container = []

In [ ]:
container.append (trainer.evaluate())

### Model Roberta-base

In [ ]:
container

[{'epoch': 4.0,
  'eval_accuracy': 0.5020297699594046,
  'eval_loss': 1.8328235149383545,
  'eval_runtime': 6.2184,
  'eval_samples_per_second': 475.363},
 {'epoch': 3.0,
  'eval_accuracy': 0.5338294993234101,
  'eval_loss': 1.8442087173461914,
  'eval_runtime': 6.2477,
  'eval_samples_per_second': 473.131},
 {'epoch': 2.0,
  'eval_accuracy': 0.5294316644113667,
  'eval_loss': 1.910200834274292,
  'eval_runtime': 6.1654,
  'eval_samples_per_second': 479.447}]

In [ ]:
from pandas import DataFrame
df = DataFrame (container,columns=['epoch','eval_accuracy','eval_loss','eval_runtime','eval_samples_per_second'])

In [ ]:
df['learning_rat'] = [0.00005, 0.00003, 0.00002] 

In [ ]:
df

,epoch,eval_accuracy,eval_loss,eval_runtime,eval_samples_per_second,learning_rat
0,4.0,0.502030,1.832824,6.2184,475.363,0.00005
1,3.0,0.533829,1.844209,6.2477,473.131,0.00003
2,2.0,0.529432,1.910201,6.1654,479.447,0.00002


### Model bert-base-cased

In [ ]:
container

[{'epoch': 2.0,
  'eval_accuracy': 0.4553450608930988,
  'eval_loss': 1.962475061416626,
  'eval_runtime': 6.9629,
  'eval_samples_per_second': 424.534},
 {'epoch': 3.0,
  'eval_accuracy': 0.5679972936400541,
  'eval_loss': 1.5792205333709717,
  'eval_runtime': 6.9238,
  'eval_samples_per_second': 426.933},
 {'epoch': 4.0,
  'eval_accuracy': 0.5720568335588633,
  'eval_loss': 1.7015208005905151,
  'eval_runtime': 6.9331,
  'eval_samples_per_second': 426.36}]

In [ ]:
from pandas import DataFrame
df = DataFrame (container,columns=['epoch','eval_accuracy','eval_loss','eval_runtime','eval_samples_per_second'])

In [ ]:
df['learning_rat'] = [0.00002, 0.00003, 0.00005] 

In [ ]:
df

,epoch,eval_accuracy,eval_loss,eval_runtime,eval_samples_per_second,learning_rat
0,2.0,0.455345,1.962475,6.9629,424.534,0.00002
1,3.0,0.567997,1.579221,6.9238,426.933,0.00003
2,4.0,0.572057,1.701521,6.9331,426.360,0.00005


# Your work

Qs 1. Can you report how accuracy changes with varying the hypyerparameters defined above? How does accuracy change with increasing number of epochs? Learning rate? Mention your final choice of parameters and graph how they impact accuracy. 

Qs 2. Try any 2 other models for sequence classification supported by Huggingface and repeat Qs 1 for these. You can find a full list of these [here](https://huggingface.co/transformers/model_doc/auto.html#automodelforsequenceclassification). (For those of you working with text for your project, you may also find several other useful models for Seq2Seq modeling, question answering, etc.)

Qs 3. Wrangle / change the data so that you can instead build a "positive" emotion vs "negative" emotions classifier, by combining labels. For any emotions that you feel are too neutral, you may drop them from evaluation. Can you now report the accuracy of a classifer on this data? You may use one of the model architectures from (1) or (2)

Qs 4. Export models from qs 1 and 3, and build a FastAPI around it, as discussed in class, which returns the most likely emotion found. (You may need to look up Huggingface documentation on how to save and load models). In particular, you should support two API endpoints - "detect_emotion_binary" and "detect_emotion_full" which supports each model.

Qs 5. Finally, can you dockerize this? You do NOT need to deploy this to the cloud.

Submission instructions: Zip your compiled .ipynb notebook, your fast_api code and your dockerfile, and submit to LMS as one file. You should confirm that your .ipynb can be imported to colab and run in sequence. All your results for Qs 1-3 should be reported in the notebook itself.